# Laboratorium 1 - A* vs Dijkstra
### Lukasz Fabia 272724


### Jak zacząć

Python3, najlepiej wersja >3.11.

```bash
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
```

## Teoria i cel

Celem tego zdania jest znalezienie najkrótszego (min. droga) przejazdu MPK lub przejechanie z punktu A do B w najkrótszym czasie (min. czasu).

Do tego problemu optymalizacji należy wykorzystać dwa popularne algorytmy do wyszukiwania najktrótszych ścieżek - A* i Dijkstra. 

---

### A*

Wykorzystuje heurystyki pomocne w wybieraniu ścieżek, mając dane koordynatów, czasu możemy utworzyć proste heurystyki, aby obliczyć koszt przejścia z jednego wierzchołka na drugi.

### Dijkstra

**Algorytm zachłanny**, znajdujący najkrótszą ścieżkę od węzła startowego do wszystkich innych węzłów w grafie. Wykorzstuje wagi (u mnie i w zadaniu korzysta on z kosztu czasu).

## Jak zbudowałem strukturę grafu

Generalnie w `/models/graph.py` mamy całą strukturę składa się ona z `Node`, `Edge` i coś co agreguje w sobie węzły, czyli graf. W klasie grafu znajduje się tylko słownik (ulica, węzeł). 

Kolejnym krokiem było parsowanie danych i wrzucenie ich do grafu. Iterowałem po wierszach i:

1. Wyciągałem dane z wiersza i robiłem z tego krawędź.

2. Dodawałem startowy i końcowy przystanek do słownika. Dodawanie działa w taki sposób, że jeśli ulicy nie ma w kluczach to dodaje dane tej ulicy pod tym kluczem.

3. Aktualizacja krawędzi, czyli do listy startowego węzła dodaje nowy edge z podpiętymi węzłami.


W ten sposób, wystarczy utworzyć obiekt i otrzymujemy `Graf skierowany ważony`.

## Wgląd do danych

Pierwsze 5 wierszy. Z racji tego, że jednym z czynników, którym będziemy się kierować przy wybieraniu trasy będzie czas no to warto różnicę, która będzie kosztem danej ścieżki.

In [1]:
from parser import get_df
df = get_df()

df.head()

,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


### Zadanie 1a 

Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu Dijkstry w oparciu o kryterium czasu.

#### Wnioski

1. Minimalny czas podróży
    
    - gwarantuje nam, że koszt faktycznie będzie najmniejszy w moim przypadku - czas podróży, ale gdy warunki będą idealne tj. brak opóźnień tramwajów.

    - można pomyśleć nad algorytmem, który obsłuży sytuacje losowe.

2. Liczba odwiedzonych wierzchołków

    - jest ona całkiem spora, gdy w gre wchodzi trudniejsza trasa na odcinku **Wyszyńskiego** - **Wielka** odwiedził aż _10323_.

3. Performance

    - długi czas wykonania się dla tras gdzie mamy kilka przystanków to kilkanaście ms, ale liczba rośnie w przypadku trudniejszych odcinków. Jest to spowodowane przeszukiwaniem wszerz co też nijako wiąże się z dużą ilością odwiedzonych wierzchołków. 

    - sam język nie jest demonem prędkości, można przepisać na coś szybszego jak (np. Rust, C/C++).

4. Optymalizacje

    - użycie algorytmu z heurystyką np. A*


In [2]:
from parser import to_graph
from dijkstra import Dijkstra
from a import AStar
from datetime import time
g = to_graph()

# times 

t1 = time(hour=8, minute=18, second=0)
t2 = time(hour=20, minute=50, second=0)
t3 = time(hour=7, minute=20, second=0)
d_engine = Dijkstra(g)

d_engine.search("Wyszyńskiego", "Wielka", t1)

d_engine.search("Zajezdnia Obornicka", "Bałtycka", t2)

d_engine.search("Wyszyńskiego", "PL. GRUNWALDZKI", t3)


Results for Dijkstra:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
--------------------------------------------------------------------------------
904   08:18:00    Wyszyńskiego                  08:20:00    Ogród Botaniczny               2         
904   08:20:00    Ogród Botaniczny              08:21:00    Katedra                        3         
904   08:21:00    Katedra                       08:22:00    Urząd Wojewódzki (Muzeum N     4         
904   08:22:00    Urząd Wojewódzki (Muzeum N    08:25:00    GALERIA DOMINIKAŃSKA           7         
D     08:28:00    GALERIA DOMINIKAŃSKA          08:32:00    Renoma                         14        
D     08:32:00    Renoma                        08:34:00    Arkady (Capitol)               16        
6     08:34:00    Arkady (Capitol)              08:36:00    Zaolziańska                    18        
6     08:36:00    Zaolziańska                   08:37:00    Wielka               

### Zadanie 1b 
Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu A* w oparciu o kryterium czasu.

#### Wnioski

1. Minimalny czas podróży

    - rozwiązanie jest gorsze jak można było się spodziewać, ale bardzo szybko je dostajemy

    - warto dodać, że jest to zależne od heurystyki
    

2. Liczba odwiedzonych wierzchołków

    - liczba jest o wiele mniejsza od rozwiązania problemu za pomocą **Dijkstry**

    - jest to spowodowane użyciem heurystyki, która przeszukuje rokujące węzły

    - przy trasie najdłuższej odwiedzonych było **735** wierzchołków, gdzie w poprzednim teście przy wykorzystaniu **Dijkstry** było to _10323_.


3. Performance

    - tu jest znaczenie lepiej bo nie przeszukujemy wszystkiego

4. Opytmalizacje

    - można dodać heurystyke, która by sprawdzała kierunek, którym ma się kierować na podstawie kątów obliczanych z koordynatów punktu A i B.   

In [3]:
a_engine = AStar(g)

a_engine.search("Wyszyńskiego", "Wielka", t1)
a_engine.search("Zajezdnia Obornicka", "Bałtycka", t2)
a_engine.search("Wyszyńskiego", "PL. GRUNWALDZKI", t3)


Results for A*:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
--------------------------------------------------------------------------------
904   08:18:00    Wyszyńskiego                  08:20:00    Ogród Botaniczny               2         
904   08:20:00    Ogród Botaniczny              08:21:00    Katedra                        3         
904   08:21:00    Katedra                       08:22:00    Urząd Wojewódzki (Muzeum N     4         
904   08:22:00    Urząd Wojewódzki (Muzeum N    08:25:00    GALERIA DOMINIKAŃSKA           7         
K     08:25:00    GALERIA DOMINIKAŃSKA          08:29:00    DWORZEC GŁÓWNY                 11        
8     08:29:00    DWORZEC GŁÓWNY                08:31:00    DWORZEC AUTOBUSOWY             13        
15    08:33:00    DWORZEC AUTOBUSOWY            08:35:00    Sanocka                        17        
15    08:35:00    Sanocka                       08:36:00    Uniwersytet Ekonomiczny    

#### Zadanie 1c

Zmutowane wersje A*. Minimalizacja liczby przesiadek. Intuicja podpowiada, że trzeba będzie wprowadzić coś w stylu kary, dodatkowego kosztu doliczanego do ścieżki, która może i jest najkrótszą, ale wymaga przesiadki właśnie.
